In [1]:
# Standard libraries
import sys
import os
import json
import importlib
import string
import math
import re
import pprint
# Third party libraries
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

pp = pprint.PrettyPrinter(indent=2, width=100)
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
print('Plotly ver: ', plotly.__version__)
print('Pandas ver: ', pd.__version__)
pd.options.display.max_rows = 150

Plotly ver:  3.6.1
Pandas ver:  0.23.4


Load the first sheet

In [2]:
df1 = pd.read_excel("./collapsed and with nutrition.xlsx", sheet_name = 'L2_Consumers_only_g_day', header = 2)
df1.set_index(['Country', 'Pop Class', 'Foodex L1', 'Metrics', 'Survey'])  

Mean calories L1  \
Country Pop Class Foodex L1                                          Metrics     Survey                                                                 
Austria Adults    Grains and grain-based products                    A.01.000013 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                                                                     A.01.000043 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                                                                     A.01.000098 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                                                                     A.01.000168 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                                                                     A.01.000184 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                                                                     A.01.000252 Austrian Study on Nutritional Status 2010-12 - ...        159.380754   
                  Vegetables and vegetable products (including fungi A.01.000317 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000318 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000331 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000337 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000350 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000359 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000382 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000385 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000418 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000440 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000453 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                                                                     A.01.000458 Austrian Study on Nutritional Status 2010-12 - ...         54.561318   
                  Starchy roots and tubers                           A.01.000468 Austrian Study on Nutritional Status 2010-12 - ...         47.172806   
                                                                     A.01.000480 Austrian Study on Nutritional Status 2010-12 - ...         47.172806   
                  Legumes, nuts and oilseeds                         A.01.000487 Austrian Study on Nutritional Status 2010-12 - ...        216.767568   
                                                                     A.01.000491 Austrian Study on Nutritional Status 2010-12 - ...        216.767568   
                                                                     A.01.000513 Austrian Study on Nutritional Status 2010-12 - ...        216.767568   
                                                                     A.01.000527 Austrian Study on Nutritional Status 2010-12 - ...        216.767568   
                  Fruit and fruit products                           A.01.000545 Austrian Study on Nutritional Status 2010-12 - ...         62.076129   
  

In [3]:
list(df1)

['Country',
 'Survey',
 'Pop Class',
 'Foodex L1',
 'Mean calories L1',
 'Mean Carbs L1',
 'Mean Prot L1',
 'Mean Fat L1',
 'Foodex L2',
 'Metrics',
 'Mean calories L2',
 'Mean Carbs L2',
 'Mean Prot L2',
 'Mean Fat L2',
 'Nr Consumer',
 '% Consumers',
 'Mean',
 'STD',
 'P5',
 'P10',
 'Median',
 'P95',
 'P97.5',
 'P99',
 'Comment']

In [4]:
df2 = df1[['Country', 'Pop Class', 'Foodex L1', 'Mean calories L1', 'Mean Carbs L1', 'Mean Prot L1', 'Mean Fat L1', 'Nr Consumer', 'Mean']]
print(df2)

       Country Pop Class                                          Foodex L1  \
0      Austria    Adults                    Grains and grain-based products   
1      Austria    Adults                    Grains and grain-based products   
2      Austria    Adults                    Grains and grain-based products   
3      Austria    Adults                    Grains and grain-based products   
4      Austria    Adults                    Grains and grain-based products   
5      Austria    Adults                    Grains and grain-based products   
6      Austria    Adults  Vegetables and vegetable products (including f...   
7      Austria    Adults  Vegetables and vegetable products (including f...   
8      Austria    Adults  Vegetables and vegetable products (including f...   
9      Austria    Adults  Vegetables and vegetable products (including f...   
10     Austria    Adults  Vegetables and vegetable products (including f...   
11     Austria    Adults  Vegetables and vegetable p

In [5]:
for row in df2:
    print(row)

Country
Pop Class
Foodex L1
Mean calories L1
Mean Carbs L1
Mean Prot L1
Mean Fat L1
Nr Consumer
Mean


In [71]:
from numpy import array

def get_mask(df, column_name, list_value):
    if type(list_value) == list:
        mask = [x == df[column_name] for x in list_value]
    else:
        mask = [df[column_name] == list_value]
        
    mask = array(mask)
    mask = mask.any(axis=0)
    return mask
    
    
def get_max_min(list_age_group, food, df):
    
    pop_mask = get_mask(df, 'Pop Class', list_age_group)
    food_mask = get_mask(df, 'Foodex L1', food)
    full_mask = pop_mask & food_mask
    
    masked_df = df2[full_mask]
    
    list_countries = masked_df.Country.unique()
    ranked_dict = dict.fromkeys(list_countries, 0)
    amount_dict = dict.fromkeys(list_countries, 0)
    
    #build dictionary
    for index, row in masked_df.iterrows():
        ranked_dict[row.Country] = ranked_dict[row.Country] + row.Mean
        amount_dict[row.Country] = amount_dict[row.Country] + 1
    
    print(amount_dict)
        
    #get mean of means
    for keys in ranked_dict:
        ranked_dict[keys] = ranked_dict[keys] / amount_dict[keys]
    
    #sort dictionary
    ranked_dict = sorted(ranked_dict.items(), key=lambda x: x[1])
    
    return(ranked_dict)

In [72]:
ranked_dict = get_max_min(['Children', 'Adults'], 'Grains and grain-based products', df2)

{'Austria': 12, 'Belgium': 18, 'Bulgaria': 18, 'Czech Republic': 12, 'Germany': 45, 'Denmark': 36, 'Spain': 30, 'Finland': 49, 'France': 12, 'United Kingdom': 36, 'Greece': 13, 'Hungary': 6, 'Ireland': 12, 'Italy': 22, 'Latvia': 18, 'Netherlands': 30, 'Romania': 6, 'Sweden': 18}


In [70]:
print (ranked_dict)

[('Bulgaria', 28.491774623563174), ('Finland', 28.88740146534399), ('Denmark', 30.078377371841597), ('United Kingdom', 32.517277005108895), ('Germany', 33.55331797706697), ('Romania', 37.0797226887719), ('France', 38.71715190178384), ('Greece', 38.93830587186003), ('Italy', 41.993595884846975), ('Sweden', 42.26444817825435), ('Netherlands', 43.59952454277939), ('Spain', 43.832473210669306), ('Belgium', 44.28304697998397), ('Ireland', 46.513046984126134), ('Hungary', 50.23836074646374), ('Czech Republic', 53.241738432389475), ('Austria', 66.12050587762593), ('Latvia', 75.38264087256316)]
